In [3]:
#Ran on Dr. Singh's workstation. Took about 3 days to finish
#Uses the Reverse Sequential Feature Selection method from sklearn

#input: csv files of feature engineering data
#output: log file with top 20 features for predicting disc, prob, sem_gpa using RSFS


from sklearn.feature_selection import SequentialFeatureSelector #feature selection
from sklearn.linear_model import LogisticRegression #disc and prob model
from sklearn.linear_model import SGDRegressor #semester gpa model

import pickle #save models for later, no retraining

import warnings #supresses warning output

import sys #for log file output
old_stdout = sys.stdout
log_file = open('output.log', 'w')
sys.stdout = log_file

import pandas as pd
import numpy as np



X_full_df = pd.read_csv("allProcessedFeatures.csv")
X_full = X_full_df.iloc[:,1:].values #doesn't read the index column

X_gpa_df = pd.read_csv("gpaProcessedFeatures.csv")
X_gpa = X_gpa_df.iloc[:,1:].values

y_disc_df = pd.read_csv("y_disc.csv")
y_disc = y_disc_df.iloc[:,1:].values

y_prob_df = pd.read_csv("y_prob.csv")
y_prob = y_prob_df.iloc[:,1:].values

y_gpa_df = pd.read_csv("y_gpa.csv")
y_gpa = y_gpa_df.iloc[:,1:].values

#give SequentialFeatureSelector an unfitted estimator!!
#nevermind, it never got through just the discontinued after 36 hours... give it fitted then check the features individually

#discontinue
with warnings.catch_warnings(record=True):
    logreg_disc= LogisticRegression(max_iter = 150, solver = 'sag').fit(X_full, y_disc)
    sfs_backward_disc = SequentialFeatureSelector(logreg_disc, n_features_to_select=20, direction="backward").fit(X_full, y_disc)
pickle.dump(sfs_backward_disc, open('sfs_backward_disc.pkl', 'wb'))   
print("done with disc")
print("-----best features for discontinue-----")
print(sfs_backward_disc.get_feature_names_out())
    
#probation
with warnings.catch_warnings(record=True):
    logreg_prob= LogisticRegression(max_iter = 150, solver = 'sag').fit(X_full, y_prob)
    sfs_backward_prob = SequentialFeatureSelector(logreg_prob, n_features_to_select=20, direction="backward").fit(X_full, y_prob)
pickle.dump(sfs_backward_prob, open('sfs_backward_prob.pkl', 'wb'))    
print("\ndone with prob")
print("-----best features for probation-----")
print(sfs_backward_prob.get_feature_names_out())

#sem_gpa
with warnings.catch_warnings(record=True):
    sgd_short = SGDRegressor(random_state = 64, max_iter = 100).fit(X_gpa, y_gpa)
    sfs_backward_gpa = SequentialFeatureSelector(sgd_short, n_features_to_select=20, direction="backward", scoring = 'neg_root_mean_squared_error').fit(X_gpa, y_gpa)
pickle.dump(sfs_backward_gpa, open('sfs_backward_gpa.pkl', 'wb'))    
print("done with all!")
print("-----best features for semester gpa-----")
print(sfs_backward_gpa.get_feature_names_out())



sys.stdout = old_stdout
log_file.close()

KeyboardInterrupt: 